In [0]:
import pandas as pd

In [0]:
from google.colab import drive
drive.mount('/content/drive/', force_remount= True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Merged_Dataset.csv', header=None, names = ['Category', 'Data'])

In [0]:
# lower case
df.Data = df.Data.apply(lambda x: x.lower())

In [0]:
# replacing various categories to Jokes/ Non Jokes

df.Category = df['Category'].replace(to_replace={'Proverbs', 'Quotes', 'News' , 'reuters','Wiki'}, value='Non Jokes', regex=True)

# df['new'] = df.apply(lambda _: '', axis=1)

In [0]:
count = 0
for i in range(0,len(df.Data)):
  if df.Data[i].startswith('b"') or df.Data[i].startswith("b'"):
    df.Data[i] = df.Data[i][2:]

In [0]:
print(len(df.Data))

408094


In [0]:
import string

# for i in range(0,len(df.Data)):
#     str1 = ''
#     for j in df.Data[i]:
#         if j not in string.punctuation:
#             str1+=j
#         df.new[i] = str1
# count = 0
for i in range(0,len(df.Data)):
    x = df.Data[i].translate(None, string.punctuation)
    df.Data[i] = x

In [0]:
a = df[df['Data'].map(len)>220] 
df=df.drop(df.index[df['Data'].map(len)>200])

In [0]:
df.shape

(383702, 2)

In [0]:
df.to_csv('/content/drive/My Drive/Colab Notebooks/Merged_Dataset_updated.csv')

In [0]:
import numpy as np
import re
import itertools
from collections import Counter

def load_data_and_labels():
  positive_examples = list(open("/content/drive/My Drive/Colab Notebooks/positive.csv").readlines())
  positive_examples = [s.strip() for s in positive_examples]
  negative_examples = list(open("/content/drive/My Drive/Colab Notebooks/negative.csv").readlines())
  negative_examples = [s.strip() for s in negative_examples]
  x_text = positive_examples + negative_examples
  x_text = [s.split(" ") for s in x_text]
  positive_labels = [[0, 1] for _ in positive_examples]
  negative_labels = [[1, 0] for _ in negative_examples]
  y = np.concatenate([positive_labels, negative_labels], 0)
  return [x_text, y]

def pad_sentences(sentences, padding_word="<PAD/>"):
  sequence_length = max(len(x) for x in sentences)
  padded_sentences = []
  for i in range(len(sentences)):
      sentence = sentences[i]
      num_padding = sequence_length - len(sentence)
      new_sentence = sentence + [padding_word] * num_padding
      padded_sentences.append(new_sentence)
  return padded_sentences

def build_vocab(sentences):
    # Build vocabulary
    word_counts = Counter(itertools.chain(*sentences))
    # Mapping from index to word
    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    # Mapping from word to index
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return [vocabulary, vocabulary_inv]

def build_input_data(sentences, labels, vocabulary):
    x = np.array([[vocabulary[word] for word in sentence] for sentence in sentences])
    y = np.array(labels)
    return [x, y]
  
  
def load_data():
    """
    Loads and preprocessed data for the MR dataset.
    Returns input vectors, labels, vocabulary, and inverse vocabulary.
    """
    # Load and preprocess data
    sentences, labels = load_data_and_labels()
    sentences_padded = pad_sentences(sentences)
    vocabulary, vocabulary_inv = build_vocab(sentences_padded)
    x, y = build_input_data(sentences_padded, labels, vocabulary)
    return [x, y, vocabulary, vocabulary_inv]
  
x, y, vocabulary, vocabulary_inv_list = load_data()  


In [0]:
!pip install gensim
from gensim.models import word2vec
from os.path import join, exists, split
import os
import numpy as np

In [0]:
def train_word2vec(sentence_matrix, vocabulary_inv,
                   num_features=300, min_word_count=1, context=5):
    
  # Set values for various parameters
  num_workers = 2  # Number of threads to run in parallel
  downsampling = 1e-3  # Downsample setting for frequent words

  # Initialize and train the model
  print('Training Word2Vec model...')
  sentences = [[vocabulary_inv[w] for w in s] for s in sentence_matrix]
  embedding_model = word2vec.Word2Vec(sentences, workers=num_workers,
                                      size=num_features, min_count=min_word_count,
                                      window=context, sample=downsampling)

  # Saving the model for later use. You can load it later using Word2Vec.load()
  print('Saving Word2Vec model')
  embedding_model.save('word2vec_model')

  # add unknown words
  embedding_weights = {key: embedding_model[word] if word in embedding_model else
                            np.random.uniform(-0.25, 0.25, embedding_model.vector_size)
                       for key, word in vocabulary_inv.items()}
  return embedding_weights
vocabulary_inv = {key: value for key, value in enumerate(vocabulary_inv_list)}
w = train_word2vec(x, vocabulary_inv)

Training Word2Vec model...
Saving Word2Vec model


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [0]:
print(len(y))

TypeError: ignored

In [0]:
import numpy as np
# import data_helpers
# from w2v import train_word2vec

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input, MaxPooling1D, Convolution1D, Embedding
from keras.layers.merge import Concatenate
# from keras.datasets import imdb
from keras.preprocessing import sequence
np.random.seed(0)

model_type = "CNN-non-static"

# Model Hyperparameters
embedding_dim = 300
filter_sizes = (3, 8)
num_filters = 10
dropout_prob = (0.5, 0.8)
hidden_dims = 50

# Training parameters
batch_size = 64
num_epochs = 10

# Prepossessing parameters
sequence_length = 400
max_words = 5000

# Word2Vec parameters (see train_word2vec)
min_word_count = 1
context = 5


y = y.argmax(axis=0)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x = x[shuffle_indices]
y = y[shuffle_indices]
train_len = int(len(x) * 0.9)
x_train = x[:train_len]
y_train = y[:train_len]
x_test = x[train_len:]
y_test = y[train_len:]

if sequence_length != x_test.shape[1]:
    print("Adjusting sequence length for actual size")
    sequence_length = x_test.shape[1]

print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
print("Vocabulary Size: {:d}".format(len(vocabulary_inv)))

input_shape = (sequence_length,)
model_input = Input(shape=input_shape)
z = Embedding(len(vocabulary_inv), embedding_dim, input_length=sequence_length, name="embedding")(model_input)
z = Dropout(dropout_prob[0])(z)

conv_blocks = []
for sz in filter_sizes:
    conv = Convolution1D(filters=num_filters,kernel_size=sz,padding="valid",activation="relu",strides=1)(z)
    conv = MaxPooling1D(pool_size=2)(conv)
    conv = Flatten()(conv)
    conv_blocks.append(conv)
    
z = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]

z = Dropout(dropout_prob[1])(z)
z = Dense(hidden_dims, activation="relu")(z)
model_output = Dense(1, activation="sigmoid")(z)

model = Model(model_input, model_output)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

weights = np.array([v for v in w.values()])
print("Initializing embedding layer with word2vec weights, shape", weights.shape)
embedding_layer = model.get_layer("embedding")
embedding_layer.set_weights([weights])
    
model.fit(x_train, y_train, batch_size=batch_size, epochs=num_epochs,
          validation_data=(x_test, y_test), verbose=2)  



TypeError: ignored